In [1]:
using DynamicPolynomials, MosekTools, JuMP, LinearAlgebra


function solve_Lasserre(x,f,g,h,k,L,silent)
    b=L-sum(x.^2)
    g=[g;b]
    
    n=length(x)
    m=length(g)
    l=length(h)

    model=Model(optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => silent))

    u=Vector{Int64}(undef,m)
    sg=Vector{Int64}(undef,m)

    for i in 1:m
        u[i]=ceil(Int64,maxdegree(g[i])/2)
        sg[i]=binomial(k-u[i]+n,n)
    end
    println("Maximal matrix size:", sg[1])

    #v2n=reverse(monomials([0.5*(x+e);0.5*(x-e)],0:k))

    w=Vector{Int64}(undef,l)
    sh=Vector{Int64}(undef,l)

    for i in 1:l
        w[i]=ceil(Int64,maxdegree(h[i])/2)
        sh[i]=binomial(2*(k-w[i])+n,n)
    end

    v=reverse(monomials(x,0:2*k))

    @variable(model, lambda)

    wsos=f-lambda

    G=Vector{Union{VariableRef,Symmetric{VariableRef,Array{VariableRef,2}}}}(undef, m)

    for i in 1:m
        if sg[i]==1
            G[i]=@variable(model,lower_bound=0)
            wsos-=g[i]*G[i]
        else
            G[i]=@variable(model,[1:sg[i],1:sg[i]],PSD)
            for p=1:sg[i], q=p:sg[i]
                if p==q
                    wsos-=g[i]*G[i][p,q]*v[p]*v[q]
                else
                    wsos-=2*g[i]*G[i][p,q]*v[p]*v[q]
                end
            end
        end
    end

    H=Vector{Vector{VariableRef}}(undef, l)

    for i in 1:l
        H[i]=@variable(model,[1:sh[i]])
        wsos-=h[i]*sum(H[i][j]*v[j] for j=1:sh[i])
    end


    @constraint(model, coefficients(wsos).==0)
    @objective(model, Max, lambda)
    optimize!(model)

    opt_val = value(lambda)
    println("Termination status = ", termination_status(model))
    println("Optimal value = ",opt_val)
end

solve_Lasserre (generic function with 1 method)

In [25]:
using DynamicPolynomials, MosekTools, JuMP


function solve_LES(x,f,g,h,k,L,s,silent)
    b=L-sum(x.^2)
    #g=[g;b]
    
    n=length(x)
    m=length(g)
    l=length(h)
    
    
    e=ones(Int64,n)

    model=Model(optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => silent))

    u=Vector{Int64}(undef,m)
    sg=Vector{Vector{Int64}}(undef,m)

    for i in 1:m
        u[i]=ceil(Int64,maxdegree(g[i])/2)
        sg[i]=Vector{Int64}(undef,k-u[i]+1)
        for r in 0:k-u[i]
            sg[i][r+1]=binomial(k-u[i]-r+2*n,2*n)
        end
    end

    @polyvar y[1:n]# variables
    mon=reverse(monomials([x;y],0:k))
    lmon=length(mon)
    v2n=Vector{Polynomial{true,Float64}}(undef,lmon)
    for i in 1:lmon
        #v2n[i]=mon[i]([x;y]=>[0.5*(x+e);0.5*(x-e)])
        v2n[i]=mon[i]([x;y]=>[(x+e);(x-e)])
    end

    #v2n=reverse(monomials([0.5*(x+e);0.5*(x-e)],0:k))

    w=Vector{Int64}(undef,l)
    sh=Vector{Int64}(undef,l)

    for i in 1:l
        w[i]=ceil(Int64,maxdegree(h[i])/2)
        sh[i]=binomial(2*(k-w[i])+n,n)
    end

    v=reverse(monomials(x,0:2*k))

    @variable(model, lambda)

    wsos=f-lambda
    

    supp=zeros(UInt64,2*n,lmon)
    z=[x;y]

    for i in 1:lmon
        for j in 1:2*n
            supp[j,i]=degree(mon[i],variable(z[j]))
        end
    end
    
    
    v2nmod=mod.(supp,2)
    
    r=1
    q=1
    maxsize=0
    
    block_G=Vector{Vector{Vector{Vector{Int64}}}}(undef,m)
    len_block_G=Vector{Vector{Vector{Int64}}}(undef,m)
    for i in 1:m
        block_G[i]=Vector{Vector{Vector{Int64}}}(undef,k-u[i]+1)
        len_block_G[i]=Vector{Vector{Int64}}(undef,k-u[i]+1)
        for t in 0:k-u[i]
            block_G[i][t+1]=Vector{Vector{Int64}}(undef,sg[i][t+1])
            len_block_G[i][t+1]=Vector{Int64}(undef,sg[i][t+1])
            for j in 1:sg[i][t+1]
                block_G[i][t+1][j]=[]
                len_block_G[i][t+1][j]=0
                r=j

                while len_block_G[i][t+1][j] <= s-1 && r <= sg[i][t+1]
                    #if all(el->iseven(el)==true, v[:,j]+v[:,r])#
                    if norm(v2nmod[:,j]-v2nmod[:,r],1)==0
                        append!(block_G[i][t+1][j],r)
                        len_block_G[i][t+1][j]+=1
                    end
                    r+=1
                end

                q=1
                while !issubset(block_G[i][t+1][j],block_G[i][t+1][q]) && q<=j-1
                    q+=1
                end

                if q<j
                    block_G[i][t+1][j]=[]
                    len_block_G[i][t+1][j]=0
                end
                #println(block_G[i][j])
                if maxsize<len_block_G[i][t+1][j]
                    maxsize=len_block_G[i][t+1][j]
                end
            end
        end
    end
    
    println("Maximal matrix size:", maxsize)
    

    G=Vector{Vector{Vector{Union{VariableRef,Symmetric{VariableRef,Array{VariableRef,2}}}}}}(undef, m)
    
    nummat=0

    for i in 1:m
        G[i]=Vector{Vector{Union{VariableRef,Symmetric{VariableRef,Array{VariableRef,2}}}}}(undef, k-u[i]+1)
        for r in 0:k-u[i]
            G[i][r+1]=Vector{Union{VariableRef,Symmetric{VariableRef,Array{VariableRef,2}}}}(undef, sg[i][r+1])
            for j in 1:sg[i][r+1]
                if len_block_G[i][r+1][j]>0
                    nummat+=1
                    if len_block_G[i][r+1][j]==1
                        G[i][r+1][j]=@variable(model,lower_bound=0)
                        wsos-=g[i]*b^r*G[i][r+1][j]*v2n[block_G[i][r+1][j][1]]^2
                    else
                        G[i][r+1][j]=@variable(model,[1:len_block_G[i][r+1][j],1:len_block_G[i][r+1][j]],PSD)
                        for p in 1:len_block_G[i][r+1][j], q in p:len_block_G[i][r+1][j]
                            if p==q
                                wsos-=g[i]*b^r*G[i][r+1][j][p,q]*v2n[block_G[i][r+1][j][p]]*v2n[block_G[i][r+1][j][q]] 
                            else
                                wsos-=2*g[i]*b^r*G[i][r+1][j][p,q]*v2n[block_G[i][r+1][j][p]]*v2n[block_G[i][r+1][j][q]]
                            end
                        end
                    end
                end
            end
        end
    end
    
    println("Number of matrix variables:", nummat)

    H=Vector{Vector{VariableRef}}(undef, l)

    for i in 1:l
        H[i]=@variable(model,[1:sh[i]])
        wsos-=h[i]*sum(H[i][j]*v[j] for j=1:sh[i])
    end


    @constraint(model, coefficients(wsos).==0)
    @objective(model, Max, lambda)
    optimize!(model)

    opt_val = value(lambda)
    println("Termination status = ", termination_status(model))
    println("Optimal value = ",opt_val)
end

solve_LES (generic function with 1 method)

In [17]:
using DynamicPolynomials, LinearAlgebra
println("***Problem setting***")

n=5
println("Number of variables: n=",n)

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

v=reverse(monomials(x,0:4))
f=generate_random_poly(v)
#f=1.0*sum(x)

L=1

m=ceil(Int64, n)
g=Vector{Polynomial{true,Float64}}(undef,m)

randx=2*rand(n).-1# create a feasible solution
randx=sqrt(L)*rand(1)[1]*randx./norm(randx)
g[1]=1.0
for j in 2:m
    g[j]=generate_random_poly(v[2:end])
    g[j]+=0.125-g[j](x => randx) #make constraints feasible
end
#g=[1.0;x]


m=length(g)
println("Number of inequality constraints: m=",m)

l=0#ceil(Int64,n)

h=Vector{Polynomial{true,Float64}}(undef,l)
for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end
#h=[prod(x)-1.0]


l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variables: n=5
Number of inequality constraints: m=5
Number of equality constraints: l=0


In [31]:
k=4
s=21#ceil(Int64,binomial(k+n,n)/2)
silent=false
@time solve_LES(x,f,g,h,k,L,s,silent)

Maximal matrix size:21
Number of matrix variables:947
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1287            
  Cones                  : 0               
  Scalar variables       : 831             
  Matrix variables       : 117             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.02    
Problem
  Name                   :                 
  Objective sense        : max         

In [19]:
k=2
silent=false
@time solve_Lasserre(x,f,g,h,k,L,silent)

Maximal matrix size:21
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 126             
  Cones                  : 0               
  Scalar variables       : 5               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   :

In [29]:
k=3
silent=false
@time solve_Lasserre(x,f,g,h,k,L,silent)

Maximal matrix size:56
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 462             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 6               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   :

In [528]:
include("../src/InterRelax.jl")
using .InterRelax

@polyvar x_ad[1:n]# variables

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info([x;x_ad],f(x=>(x-x_ad)),[g[i](x=>(x-x_ad)) for i in 2:m],[h[i](x=>(x-x_ad)) for i in 1:l],sparse=false);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

k=5
s=30
L=1

@time opt_val1=InterRelax.RelaxDense_without_multiplier(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,L=L,solver="Mosek");

7   1.6e-09  2.1e-10  2.5e-07  -9.97e-01  1.488503326e+06   1.111559360e+00   2.1e-10  41.54 
Optimizer terminated. Time: 43.07   

Termination status = INTERRUPTED
Optimal value = 1.4885033257665914e6
 47.425709 seconds (45.48 M allocations: 5.394 GiB, 1.18% gc time)
